# FRB Pipeline Simulation

In [ ]:
%matplotlib notebook
import numpy as np
from matplotlib.pyplot import *

In [ ]:

def axis_labels(x, y, z):
    xlab = xlabel(x)
    ylab = ylabel(y)
    titles = title(z)
    legends = legend
    return xlab, ylab, titles, legends

The following steps will simulate the anologue process that occurs before the data is downsampled. Any pulsar will work for this, however, the pulsar being simulated will be made using the paramters below.

In [ ]:
#Define the parameters for the noise that will be mixed with the pulse
Cf = 1400e6 #This is the centre frequency in MHz, and where we will be viewing from
Bw = 10e6 #This is the bandwidth that we are observing with our telescope
sample_rate = 12e9 # Frequency running simulation at.
period = .005 #seconds 

With a period of 5ms, we will use a time frame with this length to simulate one pulse. The bins in this simulation is determined by the sampling rate, which can be changed above. The current value is at such that 12GB ram computerss will just be able to simulate. The higher value the sampling rate is given, the more accurate the simulation becomes, but at the expense of what the computer is able to process.

In [ ]:
#Define the timescale in which we will be observing
t = np.linspace(-.0025,.0025, period*sample_rate) #Decreasing the period for this will decrease the samples obtained

To create the pulsar, first noise is simulated using a normal distribution of values. This distribution is made such that the standard deviation is 1, and is centred around 0. 

In [ ]:
#Create the Gaussian noise profile for the pulsar
length = len(t)     #This defines the number of timesamples we have, ie the number of samples collected
mu = 0              #This indicates an offset of 0, meaning the signal is perfetly centred about 0
sigma = 1           #This idicates an std of 1
gnoise = np.random.normal(mu, sigma, length) #This is the GAussian noise for the signal
pw = .0004 #seconds
print(length)

This noise is then multiplied to a gaussian that has a standard deviation equal to that of the desired pulse width.The gaussian was made using the equation \begin{equation} e^{\frac{-1}{2}(\frac{t}{pw})^2} \end{equation}.

In [ ]:
gnoise = np.exp(.5*-(t/pw)**2)*gnoise #Define the pulse of the signal. The gaussian profile takes an input of the 

Observe now that the resulting signal looks like a Gausian distribution

In [ ]:
#Now plot the signal, and observe the pulse shape. To see the pulse without destroying the computer, 
#plot every 10000th point
figure()
plot(np.linspace(0,.005, period*sample_rate)[0::10000],gnoise[0::10000], ls='none', marker='.')
axis_labels('time(s)', 'value(unitless)', 'Gaussian noise pulse')

Now, to create the pulsar, we take the Fourier transform of the Gaussian, so that we may eliminate all but one frequency of the Gaussian

In [ ]:
#Now take the fourier transform of the pulse to get it in frequency space
fourier_pulse = np.fft.fft(gnoise)

In [ ]:
#The fourier transformed data should look like broadband noise, so define the axes properly before plotting
broad_freq = np.fft.fftfreq(length, 1/(sample_rate/1e6)) #This gives the signal in MHz, (specifically from the 1e6)

In [ ]:
broad_freq

In [ ]:
figure()
plot(broad_freq[::1000], np.abs(fourier_pulse[::1000])) #Square the abs value to get better view
axis_labels("frequency(MHz)", 'amplitude(unitless)', 'Fourier transformed  gaussian pulse')

In [ ]:
#Now we modify the noise, so that there are only +- of the same frequency in it of it. This is the pulsar!
#Everything outside the two reference frequencies is turned to zero
highpass_freq = (Cf-Bw/2.0) #This sets the centre frequency to 0, with a bandwidth deifned above
highpass_index1 = int(highpass_freq/(sample_rate/2)*length/2) #This finds the point in the array with the value
highpass_index2 = length - highpass_index1 #Same but for different point
fourier_pulse[:highpass_index1] = 0 #This is turning everything outside the two points to zero
fourier_pulse[highpass_index2:] = 0 #Same as above

#Everything outside of the two frequencies is turned to 0
nyquest_index = int(length/2) #This value is determined by the nyquist sampling theorem(helps figure out where the chinkc are)
#The following is the same as above, but now for outside the frequency values
lowpass_freq = Cf+Bw/2.0 
lowpass_index1 = int(lowpass_freq/(sample_rate/2)*length/2)
lowpass_index2 = length - lowpass_index1
fourier_pulse[lowpass_index1:nyquest_index] = 0
fourier_pulse[nyquest_index:lowpass_index2] = 0

In [ ]:
figure()
plot(broad_freq[::1000], np.abs(fourier_pulse[::1000]))
axis_labels("frequency(MHz)", 'amplitude(unitless)', 'Newly created pulsar')

In [ ]:
#Now turn the pulsar back into time space from frequency space
blimited_pulse = np.fft.ifft(fourier_pulse)


In [ ]:
#If this pulse is plotted again, it sill looks like the same as before
figure()
plot(np.linspace(0,.005, period*sample_rate)[0::10000], blimited_pulse[::10000])
axis_labels('time(s)', 'value(unitless)', 'Pulsar wave packet')

In [ ]:
#Now need to create the mixing signal:
mixing_signal = np.exp(-2.00j*np.pi* Cf/sample_rate *np.arange(length)) #With this we create the mixing signal using the euler equation for the signal

#We assume the pulse is always centred around the same point, so the only part of the signal equaiton needed is 
#omega*t. 

In [ ]:

#Mix and sample down the pusar signal so that we can have an I Q data set for the bandwidth
mixed_down_s = blimited_pulse*mixing_signal
N_cutoff = int(Bw/2/sample_rate*length) #This is from the lowpass fileter definition
fmixed = np.fft.fft(mixed_down_s) #This creates the fourier transform of the mixed signal
fmixed[N_cutoff:-N_cutoff] = 0.0 # this makes it so that the imaginary magnitudes are comparable to the reals. This is
                                    #also our filter
filtered_mixed_down_s = np.fft.ifft(fmixed) #This brings the fourier space pulse back into a packet
#Downsample so that is...
downsampled_filtered_mixed_down_s = filtered_mixed_down_s[::int(sample_rate/(Bw))]
#complex sampled at 10MHz instead of 12GHz
#This gives us 50000 indicies instead of 60000000

In [ ]:
#The simulation can be run from this point

# Use gaussian noise, filter with same time width.
#Now plot the signal, and observe that the pulse shape is the same, but now using smaller increments between samples
figure()
plot(np.linspace(0,.005, len(downsampled_filtered_mixed_down_s))[0::100], downsampled_filtered_mixed_down_s[::100].real)
plot(np.linspace(0,.005, len(downsampled_filtered_mixed_down_s))[0::100], downsampled_filtered_mixed_down_s[::100].imag)
axis_labels('Time(s)', 'Amplitude', 'Downsampled Pulsar')

In [ ]:
#We repeat the pulse to get about 0.15 seconds of data
downsampled_filtered_mixed_down_s_repeated = np.tile(downsampled_filtered_mixed_down_s, 30)

In [ ]:
#This is where the pulsar is dispersed
#We do a convolution in fourier space, which must be long wrt the dispersion delay time
dis_length = len(downsampled_filtered_mixed_down_s_repeated) #This gives us the number of time samples of the 
                                                             #downsampled repeated pulse
freq_step = np.linspace(-Bw/2, Bw/2, dis_length) #This is the new frequency step for the pulse
DM = 600 #This is the dispersion measure and it is completely arbitrary
#Create the signal to be convolved with the data. This signal is the pulsar
H = np.exp(2j*np.pi*4.15e15*DM*freq_step**2/((Cf+freq_step)*Cf**2)) #Where the part that looks similar to the delay 
#time is the 'time' portion of the equation

In [ ]:
#Check how long the dispersion is
delay_time = 4.15e15 * (1/(Cf-Bw/2)**2 - 1/(Cf+Bw/2)**2) * DM
#The delay time must be significantly less than the dispersion of the data
print(delay_time) #This time must be less thah H

In [ ]:
dispersed_downsampled_filtered_mixed_sig = np.fft.ifft(np.fft.fft(downsampled_filtered_mixed_down_s_repeated)*np.fft.fftshift(H))
#The pulse signal and the dispersion of the pulse were convolved to give a dispersed pulse

In [ ]:
#Plotting the convolved signal...
figure()
plot(np.linspace(0,.15, dis_length)[::100], dispersed_downsampled_filtered_mixed_sig[::100].real)
plot(np.linspace(0,.15, dis_length)[::100], dispersed_downsampled_filtered_mixed_sig[::100].imag)
axis_labels('time(s)', 'Amplitude', 'Dispersed downsampled mixed pulsar')

In [ ]:
#now that we have the pulse, we proceed to take the fourier transform of it, to graphically show 
#that we have created what we expected. This is step two in the flowgraph on Gnu
        

In [ ]:
#We want to show how the amplitude of the pulse changes with respect to frequency and time. Currently, we can only 
#show either or

In [ ]:
#We want to intergrate to 0.1ms since this is sgnifiantly less than 5ms
nfreq = 400                                                #This is an educated, but arbitrary choice for the parameter
print(len(dispersed_downsampled_filtered_mixed_sig)/nfreq) #This gives us the frequency channels we are using

In [ ]:
#We want a 10Msps, since this is the bandwidth
times = 1/10e6

In [ ]:
len(dispersed_downsampled_filtered_mixed_sig)

In [ ]:

#To check that this number of frequencies, integrations, and sampling rate work
#integrate 5 samples
step = 400 #This comes from the number of frequency channels desired
integration_size = int(2e-4/(times)/nfreq) #This comes from the 0.2ms integration that we desire. It is .2 bc .1 was too small
n_time = int(len(dispersed_downsampled_filtered_mixed_sig)/step/integration_size) #This finds the number of
                                                            #timesteps
rec_spect = np.zeros(((n_time), step), dtype=complex) #This the is the recieved 3d plot of frequency vs time
for i in range(n_time): #This is the number of 'rows'
    for j in range(integration_size):
        spect = np.fft.fft(dispersed_downsampled_filtered_mixed_sig[(integration_size*i+j)*step : (integration_size*i+j+1)*step])
        #This is an easy way to get the row size that we desire
        rec_spect[i] += spect*spect.conjugate() #The first timestep of the frequencies needs to be multiplied by its conjugate
        #The spetra are added together because we are 'folding' the amplitudes together. If we don't, then there appears 
        #a jump in the data.

        
        
    

In [ ]:
n_frequencies = np.fft.fftfreq(n_time, 1/(10e6/1e6)) #This is the number of frequency channles. This will lable the 
                                                     #the x axis
pulse_freqs = np.fft.fftshift(n_frequencies)+Cf/1e6 #This will place the centre frequency of the plot at 1400MHz
dispersed = np.flip(np.transpose(np.fft.fftshift(abs(rec_spect))), 0)#The array of the data must be flipped to match 
                                    #the units in the plot,and it must be transposed so that time is on the x axis.

In [ ]:
dispersed.shape

In [ ]:
#This is the graph of the pulsar, where we can see that it is dispersed
figure()
imshow(dispersed, extent=[np.amin(np.linspace(0,.15, dis_length)),\
                            np.amax(np.linspace(0,.15, dis_length)),\
                            np.amin(pulse_freqs),\
                            np.amax(pulse_freqs)],\
                              aspect='auto')
axis_labels('time(s)','Frequency (MHz)',  'Dispersed Pulsar')
colorbar(label = 'power')

#Congrats!, we now have a dispersed pulsar


In [ ]:
DMs = np.arange(400,800) #Assume that we do not know the DM for the pulsar. We must then Guess a range of DM

t1s = (np.linspace(0,.15, dis_length/step/integration_size))#Define time step that the pulse arrives at
time_change = np.zeros((len(DMs), len(t1s))) #This is the time after the first frequency which the second frequncy arrives
for i in range(len(DMs)):
    change = t1s+4.15e6*DMs[i]*( 1/(np.min(pulse_freqs))**2 - 1/( pulse_freqs )**2) #This is the definition for the time that the second freq arrives
    time_change[i,:] = change[:]


In [ ]:
dedisperse = np.zeros((len(DMs),dispersed.shape[0], dispersed.shape[1]))  #The dedispersed pulse will have the same shape as the dispersed one
for i in range(len(DMs)):
    dedis = dispersed*0
    for j in range(step):
        de = np.roll(dispersed[j,:],-int(time_change[i,j]*(n_time/step*integration_size))) #np.roll shifts 
                #the data over after the last position and reintroduces it to the first position
        dedis[j,:] = de[:]
    dedisperse[i,:,:] = dedis

In [ ]:
figure()
imshow(dedisperse[200], extent=[np.amin(np.linspace(0,.15, dis_length)),\
                            np.amax(np.linspace(0,.15, dis_length)),\
                            np.amin(pulse_freqs),\
                            np.amax(pulse_freqs)],\
                              aspect='auto')
axis_labels('time(s)','Frequency (MHz)',  'Dedispersed Pulsar (DM=600)')
colorbar(label = 'power') #This shows the effec the dedispersion had on the 200th DM
savefig("/home/andy/Pictures/Dedispersed_pulsar.png")

In [ ]:
DM_measure = np.sum(dedisperse, axis = 1)

In [ ]:
figure()
imshow((abs(DM_measure)), extent=[np.amin(np.linspace(0,.15, dis_length)),\
                            np.amax(np.linspace(0,.15, dis_length)),\
                            np.amin(DMs),\
                            np.amax(DMs)],\
                              aspect='auto')
axis_labels('time(s)','Dispersion Measure',  'Dispersion Measure vs Time of Pulsar (DM=600)')
colorbar(label = 'power') #This is the plot of DM vs time
savefig("/home/andy/Pictures/dispersion_measure_vs_time_DM=600.png")

In [ ]:
summed_pulse = np.sum(dedisperse, axis=0)

In [ ]:
figure()
plot(summed_pulse)

In [ ]:
.15/25
